### Intensité et porportion des lésions (01 and 07 -> 13)

In [3]:
import SimpleITK as sitk
import numpy as np
import csv
from pathlib import Path

data_path = Path.cwd().parent.parent / "data"
mslesseg_path = data_path / "01_MSLesSeg_Dataset"
registration_path = data_path / "07_registered_aseg_nii_gz_dir"

if not registration_path.exists():
    raise FileNotFoundError(f"You must run first the script of 'aseg_t1_registration.py' to generate the registered data")

proportion_dir_path = data_path / "13_lesion_proportion_with_intensity_csv_dir"
if not proportion_dir_path.exists():
    proportion_dir_path.mkdir(parents = True, exist_ok = True)

In [4]:
"""
01 and 07 -> 13
Ce script prend envrion 22 minutes.
"""

# =============================
# LUT FREESURFER ASEG
# =============================
freesurfer_aseg_labels = {

    0: "Background",

    2: "Left-Cerebral-White-Matter",
    3: "Left-Cerebral-Cortex",
    4: "Left-Lateral-Ventricle",
    5: "Left-Inf-Lat-Vent",

    7: "Left-Cerebellum-White-Matter",
    8: "Left-Cerebellum-Cortex",

    10: "Left-Thalamus-Proper",
    11: "Left-Caudate",
    12: "Left-Putamen",
    13: "Left-Pallidum",

    14: "3rd-Ventricle",
    15: "4th-Ventricle",
    16: "Brain-Stem",

    17: "Left-Hippocampus",
    18: "Left-Amygdala",

    24: "CSF",
    26: "Left-Accumbens-area",
    28: "Left-VentralDC",

    30: "Left-vessel",
    31: "Left-choroid-plexus",

    41: "Right-Cerebral-White-Matter",
    42: "Right-Cerebral-Cortex",
    43: "Right-Lateral-Ventricle",
    44: "Right-Inf-Lat-Vent",

    46: "Right-Cerebellum-White-Matter",
    47: "Right-Cerebellum-Cortex",

    49: "Right-Thalamus-Proper",
    50: "Right-Caudate",
    51: "Right-Putamen",
    52: "Right-Pallidum",

    53: "Right-Hippocampus",
    54: "Right-Amygdala",

    58: "Right-Accumbens-area",
    60: "Right-VentralDC",

    62: "Right-vessel",
    63: "Right-choroid-plexus",

    72: "5th-Ventricle",
    77: "WM-hypointensities",

    80: "non-WM-hypointensities",
    85: "Optic-Chiasm",

    251: "CC_Posterior",
    252: "CC_Mid_Posterior",
    253: "CC_Central",
    254: "CC_Mid_Anterior",
    255: "CC_Anterior",
}

# =============================
# INDEXATION DES FICHIERS
# =============================

aseg_split_path_dict = {}
t1_split_path_dict = {}

for aseg_path in registration_path.rglob("*_aseg.nii.gz"):
    parts = aseg_path.stem.split("_")
    arg = tuple(parts[:2]) if len(parts) == 3 else parts[0]
    aseg_split_path_dict[arg] = aseg_path


for t1_path in mslesseg_path.rglob("*_T1.nii.gz"):
    parts = t1_path.stem.split("_")
    arg = tuple(parts[:2]) if len(parts) == 3 else parts[0]
    t1_split_path_dict[arg] = t1_path


# =============================
# BOUCLE PRINCIPALE
# =============================

for split in aseg_split_path_dict.keys():

    if split not in t1_split_path_dict:
        raise ValueError(f"T1 manquant pour {split}")


    # Infos patient
    if isinstance(split, tuple):
        patient, timepoint = split
    else:
        patient = split
        timepoint = None


    # Fichiers
    aseg_path = aseg_split_path_dict[split]
    t1_path = t1_split_path_dict[split]

    t2_path = t1_path.parent / t1_path.name.replace("_T1.nii.gz", "_T2.nii.gz")
    flair_path = t1_path.parent / t1_path.name.replace("_T1.nii.gz", "_FLAIR.nii.gz")
    mask_path = t1_path.parent / t1_path.name.replace("_T1.nii.gz", "_MASK.nii.gz")


    modalities = {
        "T1": t1_path,
        "T2": t2_path,
        "FLAIR": flair_path,
    }


    # =============================
    # LECTURE IMAGES
    # =============================

    mask_img = sitk.Cast(
        sitk.ReadImage(str(mask_path)) > 0,
        sitk.sitkUInt8
    )

    aseg_img = sitk.ReadImage(str(aseg_path))


    mask_cc = sitk.RelabelComponent(
        sitk.ConnectedComponent(mask_img)
    )


    mask_array = sitk.GetArrayFromImage(mask_cc)
    aseg_array = sitk.GetArrayFromImage(aseg_img)


    spacing = mask_img.GetSpacing()
    voxel_volume = spacing[0] * spacing[1] * spacing[2]


    num_lesions = int(mask_array.max())

    print(f"[{patient} {timepoint}] Lésions : {num_lesions}")


    # =============================
    # PROPORTIONS ANATOMIQUES
    # =============================

    lesion_zone_proportions = {}

    unique_zones = np.unique(aseg_array)


    for lesion_id in range(1, num_lesions + 1):

        lesion_mask = mask_array == lesion_id
        total_voxels = lesion_mask.sum()

        zone_props = {}

        for zone in unique_zones:
            n = np.logical_and(lesion_mask, aseg_array == zone).sum()

            if n > 0:
                zone_name = freesurfer_aseg_labels.get(zone, f"Zone_{zone}")
                zone_props[f"{zone_name}_prop"] = round(n / total_voxels, 3)


        lesion_zone_proportions[lesion_id] = zone_props


    # =============================
    # HEADER CSV
    # =============================

    zone_columns = sorted({
        k for props in lesion_zone_proportions.values()
        for k in props.keys()
    })


    header = [
        "patient",
        "timepoint",
        "lesion_id",
        "lesion_volume_mm3",

        "lesion_mean",
        "lesion_std",

        "lesion_min",
        "lesion_max",
        "lesion_range",

        "lesion_skew",
        "lesion_kurt",

    ] + zone_columns


    # =============================
    # CSV PAR MODALITÉ
    # =============================

    for modality, img_path in modalities.items():


        # Nom fichier
        if timepoint is None:
            csv_name = f"{patient}_{modality}_lesion_proportion.csv"
            timepoint = "T1"
        else:
            csv_name = f"{patient}_{timepoint}_{modality}_lesion_proportion.csv"
        csv_path = proportion_dir_path / csv_name


        with open(csv_path, "w", newline="") as f:

            writer = csv.DictWriter(f,fieldnames = header)

            writer.writeheader()


            # Lecture image
            img = sitk.Cast(
                sitk.ReadImage(str(img_path)),
                sitk.sitkFloat64
            )

            img_array = sitk.GetArrayFromImage(img)


            # =============================
            # PARCOURS LÉSIONS
            # =============================

            for lesion_id in range(1, num_lesions + 1):

                lesion_mask = mask_array == lesion_id
                voxels = img_array[lesion_mask]


                mean = round(voxels.mean(), 1)
                std = round(voxels.std(), 1)

                vmin = round(voxels.min(), 1)
                vmax = round(voxels.max(), 1)


                if voxels.size >= 3 and std > 0:
                    centered = voxels - mean
                    skew = round(np.mean(centered**3) / std**3, 2)
                    kurt = round(np.mean(centered**4) / std**4 - 3, 2)
                else:
                    skew, kurt = "", ""

                row = {

                    "patient": patient,
                    "timepoint": timepoint,
                    "lesion_id": lesion_id,

                    "lesion_volume_mm3": round(lesion_mask.sum() * voxel_volume, 1),

                    "lesion_mean": mean,
                    "lesion_std": std,

                    "lesion_min": vmin,
                    "lesion_max": vmax,

                    "lesion_range": round(vmax - vmin, 1),

                    "lesion_skew": skew,
                    "lesion_kurt": kurt,
                }

                # Proportions
                for col in zone_columns:
                    row[col] = lesion_zone_proportions[lesion_id].get(col, 0)

                writer.writerow(row)

        print(f"CSV généré : {csv_name}")

print("Traitement terminé.")

[P10 T1] Lésions : 35
CSV généré : P10_T1_T1_lesion_proportion.csv
CSV généré : P10_T1_T2_lesion_proportion.csv
CSV généré : P10_T1_FLAIR_lesion_proportion.csv
[P10 T2] Lésions : 32
CSV généré : P10_T2_T1_lesion_proportion.csv
CSV généré : P10_T2_T2_lesion_proportion.csv
CSV généré : P10_T2_FLAIR_lesion_proportion.csv
[P11 T1] Lésions : 15
CSV généré : P11_T1_T1_lesion_proportion.csv
CSV généré : P11_T1_T2_lesion_proportion.csv
CSV généré : P11_T1_FLAIR_lesion_proportion.csv
[P11 T2] Lésions : 10
CSV généré : P11_T2_T1_lesion_proportion.csv
CSV généré : P11_T2_T2_lesion_proportion.csv
CSV généré : P11_T2_FLAIR_lesion_proportion.csv
[P12 T1] Lésions : 36
CSV généré : P12_T1_T1_lesion_proportion.csv
CSV généré : P12_T1_T2_lesion_proportion.csv
CSV généré : P12_T1_FLAIR_lesion_proportion.csv
[P12 T2] Lésions : 42
CSV généré : P12_T2_T1_lesion_proportion.csv
CSV généré : P12_T2_T2_lesion_proportion.csv
CSV généré : P12_T2_FLAIR_lesion_proportion.csv
[P12 T3] Lésions : 44
CSV généré : P12_T